In [2]:
import numpy as np
import sys
import cv2
import torch as th
import copy
import blobfile as bf
import PIL
import torchvision
sys.path.insert(1, '/home/mint/guided-diffusion/sample_scripts/sample_utils/')
import mani_utils, inference_utils, params_utils
sys.path.insert(1, '/home/mint/guided-diffusion/guided_diffusion/dataloader/')

def load_image(path):
    with bf.BlobFile(path, "rb") as f:
        pil_image = PIL.Image.open(f)
        pil_image.load()
    pil_image = pil_image.convert("RGB")
    return pil_image

def manual_sh(sh, basis_idx, n_step):
    sh_in = sh.reshape(1, 9, 3)
    # print("GG", sh_in[0:1, [0].shape)
    sh_out = []
    for i in range(9):
        if i in basis_idx:
            # x 
            sh_x = np.linspace(0, 5, num=n_step)
            # y 
            sh_y = np.linspace(0, 5, num=n_step)
            # z 
            sh_z = np.linspace(0, 5, num=n_step)
            tmp = np.stack((sh_x, sh_y, sh_z), axis=-1)
            tmp = tmp[:, None, :]
        else:
            tmp = np.repeat(a=sh_in[0:1, [i], :3], repeats=n_step, axis=0)
        sh_out.append(tmp)
    sh_out = np.concatenate(sh_out, axis=1)
    return sh_out.reshape(n_step, -1)

cond, _ = params_utils.load_params(path='/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/', params_key=['shape', 'pose', 'exp', 'cam', 'light', 'detail', 'albedo'])


Key=> shape : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-shape-anno.txt
Key=> pose : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-pose-anno.txt
Key=> exp : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-exp-anno.txt
Key=> cam : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-cam-anno.txt
Key=> light : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-light-anno.txt
Key=> detail : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-detail-anno.txt
Key=> albedo : Filename=>/data/mint/DPM_Dataset/ffhq_256_with_anno/params/valid/ffhq-valid-albedo-anno.txt


In [3]:
#NOTE: Initialize a DECA renderer
sj = copy.deepcopy(cond['66646.jpg'])
sj['raw_image_path'] = ['/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/valid/66646.jpg']
raw_img = load_image(sj['raw_image_path'][0])
raw_img = th.tensor(np.array(raw_img)[None])
raw_img = raw_img.permute(0, 3, 1, 2)
sj['raw_image'] = raw_img

for k in ['pose', 'shape', 'exp', 'cam', 'light', 'albedo', 'detail']:
    sj[k] = th.tensor(sj[k][None])
mask = params_utils.load_flame_mask()
deca_obj = params_utils.init_deca(mask=mask)

n_step = 100
sh_mod = grid_sh(sj['light'], basis_idx=[0], n_step=n_step)
sj['light'] = th.tensor(sh_mod)
# sj['R_normals'] = params_utils.get_R_normals(n_step=n_step)
deca_rendered, _ = params_utils.render_deca(deca_params=sj, 
                                            idx=0, n=n_step, 
                                            avg_dict=None, 
                                            render_mode='shape', 
                                            rotate_normals=False, 
                                            mask=mask,
                                            deca_obj=deca_obj)

deca_rendered = (deca_rendered.permute(0, 2, 3, 1).mul(255).add_(0.5).clamp_(0, 255)).type(th.ByteTensor)
torchvision.io.write_video(video_array=deca_rendered, filename=f"./ren.mp4", fps=30)

creating the FLAME Decoder
[#] DECA : SRenderY applied mask


/home/mint/guided-diffusion/sample_scripts/sample_utils/params_utils.py:146: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'light':th.tensor(deca_params['light']).to(device).reshape(-1, 9, 3).float(),
